<a href="https://colab.research.google.com/github/minhaz1172/NLP/blob/main/Text_Classification(Using_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We will be using a dataset called "Economic news article tone and relevance" which consists of approximately 8000 news articles, which were tagged as relevant or not relevant to the US Economy. Our goal in this kernel is to explore the process of training and testing text classifiers for this dataset.

In [3]:
import numpy as np
import pandas as pd

# import dataset from kaggle

In [4]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"minhaz1172","key":"c20c43ca9e6e5be6fa69c912fab503c3"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!pip install kaggle

In [7]:
!kaggle datasets download -d heeraldedhia/us-economic-news-articles
!unzip us-economic-news-articles.zip -d data


Dataset URL: https://www.kaggle.com/datasets/heeraldedhia/us-economic-news-articles
License(s): CC0-1.0
us-economic-news-articles.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  us-economic-news-articles.zip
replace data/US-Economic-News.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [8]:
df=pd.read_csv("data/US-Economic-News.csv", encoding='latin-1')
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


# separating target and feature

In [9]:
data = df.drop(['_unit_id','_golden', '_unit_state','_trusted_judgments','_last_judgment_at','positivity','positivity:confidence','relevance:confidence',	'articleid','date','headline','positivity_gold','relevance_gold'], axis=1)
data.head()

,relevance,text
0,yes,NEW YORK -- Yields on most certificates of dep...
1,no,The Wall Street Journal Online</br></br>The Mo...
2,no,WASHINGTON -- In an effort to achieve banking ...
3,no,The statistics on the enormous costs of employ...
4,yes,NEW YORK -- Indecision marked the dollar's ton...


## swapping text and relevance

In [10]:
data = data[['text', 'relevance']]
data.head()

,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,yes
1,The Wall Street Journal Online</br></br>The Mo...,no
2,WASHINGTON -- In an effort to achieve banking ...,no
3,The statistics on the enormous costs of employ...,no
4,NEW YORK -- Indecision marked the dollar's ton...,yes


In [11]:
data.shape

(8000, 2)

In [12]:
data['text'][1]

'The Wall Street Journal Online</br></br>The Morning Brief, a look at the day\'s biggest news, is emailed to subscribers by 7 a.m. every business day. Sign up for the e-mail here.</br></br>On Friday evening, with Congress out of town on its summer recess and Americans heading into a mid-August weekend, the Bush administration sent a message to the states: The federal government will make it tougher for a national children\'s insurance program to cover the offspring of middle-income families.</br></br>The State Children\'s Health Insurance Program was created in 1997 to help children whose families couldn\'t afford insurance but didn\'t qualify for Medicaid, and administration officials tell the New York Times that the changes are aimed at returning the program to its low- income focus and assuring it didn\'t become a replacement for private insurance. Administration point man Dennis Smith wrote to state officials saying there would be new restrictions on the District of Columbia and th

In [13]:
data['relevance'].value_counts()

,count
relevance,
no,6571
yes,1420
not sure,9


In [14]:
data.isnull().sum()

,0
text,0
relevance,0


# plot a bar

In [15]:
import plotly.graph_objects as go
fig=go.Figure(go.Bar(x=data['relevance'].value_counts().index, y=data['relevance'].value_counts()))
# Customize the layout
fig.update_layout(
    title='Relevance vs Sentiment',
    xaxis_title='Relevance',
    yaxis_title='Number of SEntiments',
    # template=''  # Use a specific theme
)
fig.show()

In [16]:
data.duplicated().sum()

np.int64(6)

# Preprocessing relevance column by Labelencoder

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['relevance']=le.fit_transform(data['relevance'])
data.head()

,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,2
1,The Wall Street Journal Online</br></br>The Mo...,0
2,WASHINGTON -- In an effort to achieve banking ...,0
3,The statistics on the enormous costs of employ...,0
4,NEW YORK -- Indecision marked the dollar's ton...,2


# LowercaSE

In [18]:

data['text'] = data['text'].str.lower()
data['text']

,text
0,new york -- yields on most certificates of dep...
1,the wall street journal online</br></br>the mo...
2,washington -- in an effort to achieve banking ...
3,the statistics on the enormous costs of employ...
4,new york -- indecision marked the dollar's ton...
...,...
7995,secretary of commerce charles w. sawyer said y...
7996,"u.s. stocks inched up last week, overcoming co..."
7997,ben s. bernanke cleared a key hurdle thursday ...
7998,the white house's push to contract out many fe...


# Remove html tags

In [19]:
import re
def remove_html_tag(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

# Remove from dataset
data['text']=data['text'].apply(remove_html_tag)
data['text']

,text
0,new york -- yields on most certificates of dep...
1,the wall street journal onlinethe morning brie...
2,washington -- in an effort to achieve banking ...
3,the statistics on the enormous costs of employ...
4,new york -- indecision marked the dollar's ton...
...,...
7995,secretary of commerce charles w. sawyer said y...
7996,"u.s. stocks inched up last week, overcoming co..."
7997,ben s. bernanke cleared a key hurdle thursday ...
7998,the white house's push to contract out many fe...


# Remove Urls

In [20]:
import re
def remove_url(text):
  pattern=re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'',text)

# Remove from dataset
data['text']=data['text'].apply(remove_url)

# Remove punctuation

In [21]:
import string
def remove_punc2(text):
  tex=text.lower()
  text=text.translate(str.maketrans('','',string.punctuation))
  return text

# Remove from dataset
data['text']=data['text'].apply(remove_punc2)

# Spelling Correction

# Removing Stop Words

In [22]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [23]:
from nltk.tokenize import word_tokenize
def remove_stopwords(text):
  word_tokens=word_tokenize(text)
  filtered_text=[word for word in word_tokens if word not in stop]
  return " ".join(filtered_text)

data['text']=data['text'].apply(remove_stopwords)
data.head()

,text,relevance
0,new york yields certificates deposit offered m...,2
1,wall street journal onlinethe morning brief lo...,0
2,washington effort achieve banking reform senat...,0
3,statistics enormous costs employee drug abuse ...,0
4,new york indecision marked dollars tone trader...,2


In [24]:
data['relevance'].value_counts()

,count
relevance,
0,6571
2,1420
1,9


# Select features and Target

In [25]:
X=data.drop('relevance', axis=1)
y=data['relevance']

# Train test split

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
X_train

,text
1467,federal reserve acquired 1 trillion worth mort...
5768,1000 employes day would go strict hiring diet ...
5714,president ford drew magic marker chart represe...
1578,us economy might getting back feet quite quick...
6958,blacks hispanics washington area least twice l...
...,...
5226,mid1990 standard poorûªs corp downgraded 650 ...
5390,recent political cartoon mexico showed preside...
860,interestrate boost federal reserve appears cer...
7603,j george humphrey secretary ot treasury one ma...


# Apply Bag OF words

In [28]:
# we willl use counvetorizer
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X_train_bow=cv.fit_transform(X_train['text'])
X_test_bow=cv.transform(X_test['text'])
print(X_train_bow[:5])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 533 stored elements and shape (5, 70927)>
  Coords	Values
  (0, 28329)	2
  (0, 53596)	2
  (0, 9455)	1
  (0, 64224)	3
  (0, 68419)	1
  (0, 43477)	2
  (0, 56385)	2
  (0, 47552)	1
  (0, 1809)	1
  (0, 43249)	1
  (0, 49127)	1
  (0, 41846)	2
  (0, 18898)	1
  (0, 67222)	1
  (0, 28321)	5
  (0, 45752)	1
  (0, 64379)	1
  (0, 22276)	1
  (0, 30940)	1
  (0, 54210)	1
  (0, 68043)	1
  (0, 37048)	1
  (0, 28718)	1
  (0, 41139)	2
  (0, 44047)	1
  :	:
  (4, 47474)	1
  (4, 27579)	1
  (4, 25647)	2
  (4, 9944)	1
  (4, 34911)	1
  (4, 16743)	1
  (4, 37253)	1
  (4, 64685)	1
  (4, 49337)	1
  (4, 56467)	1
  (4, 17504)	1
  (4, 70023)	1
  (4, 21737)	1
  (4, 30810)	1
  (4, 62776)	1
  (4, 9593)	1
  (4, 38962)	1
  (4, 52201)	2
  (4, 23406)	2
  (4, 60841)	1
  (4, 27577)	1
  (4, 40590)	1
  (4, 23052)	1
  (4, 11289)	1
  (4, 44778)	1


In [29]:
feature_names=cv.get_feature_names_out()[:5]
feature_names



array(['00', '000', '0000', '00000', '000100000100'], dtype=object)

In [30]:
print(list(cv.vocabulary_.keys())[:20])

['federal', 'reserve', 'acquired', 'trillion', 'worth', 'mortgagebacked', 'securities', 'past', '15', 'months', 'policy', 'meeting', 'coming', 'week', 'fed', 'officials', 'try', 'decide', 'get', 'rid']


# Using Naive bayes

In [31]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train_bow.toarray(),y_train)

GaussianNB()

# Prediction

In [32]:
y_pred=model.predict(X_test_bow.toarray())
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.811875

In [33]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1269,    0,   34],
       [   1,    0,    0],
       [ 266,    0,   30]])

# Now we will use RandomForestClassifier

In [36]:
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier()
model2.fit(X_train_bow.toarray(),y_train)

RandomForestClassifier()

In [37]:
y_pred=model2.predict(X_test_bow.toarray())
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.81375